In [344]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from matplotlib import pyplot as plt

In [345]:
data_train = pd.read_csv('/Users/muzafarov/Desktop/MyEducation/ML_OE/adult_data_train.csv')
data_pred = pd.read_csv('/Users/muzafarov/Desktop/MyEducation/ML_OE/adult_data_reserved.csv')
data_train.shape, data_pred.shape

((26048, 15), (6513, 14))

In [346]:
data_train.replace('?', pd.NA, inplace=True)
data_pred.replace('?', pd.NA, inplace=True)


Удаляем пропущенные значения в train

In [347]:
# Удаление строк с пропущенными значениями в df_train
data_train.dropna(inplace=True)
data_train.reset_index(drop=True, inplace=True)
# data_train.fillna(data_pred.mode().iloc[0], inplace=True)
data_train.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,32,Private,37210,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,1
1,43,Private,101950,Masters,14,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,45,United-States,0
2,40,Local-gov,24763,Some-college,10,Divorced,Transport-moving,Unmarried,White,Male,6849,0,40,United-States,0
3,24,Private,113936,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,0
4,51,Private,237630,HS-grad,9,Married-civ-spouse,Tech-support,Husband,White,Male,7298,0,50,United-States,1
5,44,Private,310255,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,60,United-States,1
6,53,Private,121441,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,0,0,55,United-States,0
7,44,Self-emp-inc,191712,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,2415,55,United-States,1
8,32,Private,113364,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,40,United-States,1
9,51,Private,63424,Masters,14,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,40,United-States,1


Заменяем пропущенные значения на наиболее популярные в test

In [348]:
data_pred.fillna(data_pred.mode().iloc[0], inplace=True)
data_pred.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Self-emp-inc,117963,Doctorate,16,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States
1,20,Private,201490,Some-college,10,Never-married,Adm-clerical,Own-child,White,Male,0,0,40,United-States
2,78,Self-emp-inc,385242,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,9386,0,45,United-States
3,32,Private,164197,Bachelors,13,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States
4,61,Private,92691,HS-grad,9,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,3,United-States
5,32,Private,106541,5th-6th,3,Married-civ-spouse,Other-service,Other-relative,White,Male,0,0,40,United-States
6,33,Private,175412,9th,5,Divorced,Craft-repair,Unmarried,White,Male,114,0,55,United-States
7,39,Private,49020,Masters,14,Never-married,Exec-managerial,Not-in-family,White,Male,0,1974,40,United-States
8,49,Private,122206,Assoc-voc,11,Married-civ-spouse,Sales,Husband,White,Male,0,0,25,United-States
9,45,Private,100651,HS-grad,9,Divorced,Other-service,Not-in-family,White,Male,0,1980,40,United-States


In [349]:
for column in data_pred.columns:
    # Печатаем название текущего столбца
    print(f"Processing column: {column}")

    # Проходимся по каждому значению в текущем столбце
    for i, value in enumerate(data_pred[column]):
      if value not in data_train[column].values:
        most_frequent_value = data_pred[column].mode().iloc[0]
        data_pred.at[i, column] = most_frequent_value

data_pred.columns

Processing column: age
Processing column: workclass
Processing column: fnlwgt
Processing column: education
Processing column: education-num
Processing column: marital-status
Processing column: occupation
Processing column: relationship
Processing column: race
Processing column: sex
Processing column: capital-gain
Processing column: capital-loss
Processing column: hours-per-week
Processing column: native-country


Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')

In [350]:
data_pred.shape,data_train.shape

((6513, 14), (24134, 15))

In [351]:
le = LabelEncoder()

In [352]:
data_train.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,32,Private,37210,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,1
1,43,Private,101950,Masters,14,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,45,United-States,0
2,40,Local-gov,24763,Some-college,10,Divorced,Transport-moving,Unmarried,White,Male,6849,0,40,United-States,0
3,24,Private,113936,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,0
4,51,Private,237630,HS-grad,9,Married-civ-spouse,Tech-support,Husband,White,Male,7298,0,50,United-States,1
5,44,Private,310255,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,60,United-States,1
6,53,Private,121441,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,0,0,55,United-States,0
7,44,Self-emp-inc,191712,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,2415,55,United-States,1
8,32,Private,113364,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,40,United-States,1
9,51,Private,63424,Masters,14,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,40,United-States,1


In [353]:
column_str = ['workclass','education', 'marital-status', 'occupation', 'relationship', 'race', 'sex','native-country']
for i in column_str:
    data_train[i] = le.fit_transform(data_train[i])
data_train.head(10)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,32,2,37210,9,13,2,3,0,4,1,0,0,45,38,1
1,43,2,101950,12,14,4,3,1,4,0,0,0,45,38,0
2,40,1,24763,15,10,0,13,4,4,1,6849,0,40,38,0
3,24,2,113936,9,13,4,9,3,4,1,0,0,40,38,0
4,51,2,237630,11,9,2,12,0,4,1,7298,0,50,38,1
5,44,2,310255,15,10,2,2,0,4,1,0,0,60,38,1
6,53,2,121441,15,10,2,11,0,4,1,0,0,55,38,0
7,44,3,191712,12,14,2,9,0,4,1,0,2415,55,38,1
8,32,2,113364,9,13,4,9,1,4,1,0,0,40,38,1
9,51,2,63424,12,14,2,9,5,4,0,0,0,40,38,1


In [354]:
data_pred.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Self-emp-inc,117963,Doctorate,16,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States
1,20,Private,201490,Some-college,10,Never-married,Adm-clerical,Own-child,White,Male,0,0,40,United-States
2,78,Self-emp-inc,174789,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,9386,0,45,United-States
3,32,Private,164197,Bachelors,13,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States
4,61,Private,174789,HS-grad,9,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,3,United-States
5,32,Private,106541,5th-6th,3,Married-civ-spouse,Other-service,Other-relative,White,Male,0,0,40,United-States
6,33,Private,174789,9th,5,Divorced,Craft-repair,Unmarried,White,Male,114,0,55,United-States
7,39,Private,49020,Masters,14,Never-married,Exec-managerial,Not-in-family,White,Male,0,1974,40,United-States
8,49,Private,122206,Assoc-voc,11,Married-civ-spouse,Sales,Husband,White,Male,0,0,25,United-States
9,45,Private,174789,HS-grad,9,Divorced,Other-service,Not-in-family,White,Male,0,1980,40,United-States


In [355]:
for i in column_str:
    data_pred[i] = le.fit_transform(data_pred[i])
data_pred.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,3,117963,10,16,4,9,3,4,1,0,0,40,37
1,20,2,201490,15,10,4,0,3,4,1,0,0,40,37
2,78,3,174789,9,13,2,3,0,4,1,9386,0,45,37
3,32,2,164197,9,13,2,2,0,4,1,0,0,44,37
4,61,2,174789,11,9,2,0,0,4,1,0,0,3,37
5,32,2,106541,4,3,2,7,2,4,1,0,0,40,37
6,33,2,174789,6,5,0,2,4,4,1,114,0,55,37
7,39,2,49020,12,14,4,3,1,4,1,0,1974,40,37
8,49,2,122206,8,11,2,11,0,4,1,0,0,25,37
9,45,2,174789,11,9,0,7,1,4,1,0,1980,40,37


In [356]:
X_train = data_train.drop(columns = 'label')
y_train = data_train['label']
X_test = data_pred

In [357]:
scaler = MinMaxScaler()
scaler.fit(X_train)

MinMaxScaler()

In [358]:
X_train_scaler= scaler.transform(X_train)
X_test_scaler = scaler.transform(X_test)
X_train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,32,2,37210,9,13,2,3,0,4,1,0,0,45,38
1,43,2,101950,12,14,4,3,1,4,0,0,0,45,38
2,40,1,24763,15,10,0,13,4,4,1,6849,0,40,38
3,24,2,113936,9,13,4,9,3,4,1,0,0,40,38
4,51,2,237630,11,9,2,12,0,4,1,7298,0,50,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24129,35,4,179171,11,9,4,11,4,2,0,0,0,38,10
24130,39,2,421633,15,10,0,10,4,2,0,0,0,30,38
24131,55,2,82098,11,9,2,3,0,1,1,0,0,55,38
24132,26,2,50053,9,13,2,3,0,4,1,0,0,55,38


In [359]:
neigh = KNeighborsClassifier(n_neighbors=13)
neigh.fit(X_train_scaler, y_train)

KNeighborsClassifier(n_neighbors=13)

In [360]:
y_predict = neigh.predict(X_test_scaler)

In [361]:
y_predict.tolist()

[0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [362]:
with open('/Users/muzafarov/Desktop/MyEducation/ML_OE/adult_division.txt', 'w') as f:
    f.write(f'{list(y_predict)}')

In [363]:
from sklearn.ensemble import RandomForestClassifier

In [364]:
rfc = RandomForestClassifier()

In [365]:
rfc.fit(X_train_scaler, y_train)

RandomForestClassifier()

In [366]:
preds = rfc.predict(X_test_scaler)

In [367]:
preds

array([0, 0, 1, ..., 1, 0, 0])

In [368]:
with open('/Users/muzafarov/Desktop/MyEducation/ML_OE/adult_division_2.txt', 'w') as f:
    f.write(f'{list(y_predict)}')